In [28]:
import sys, os
import pandas as pd
import numpy as np
from sklearn import metrics
sys.path.append(os.path.abspath(os.path.join(os.path.dirname("Jaccard_similarity"), '..')))
sys.path.append(os.path.abspath(os.path.join(os.path.dirname("Jaccard_similarity"), '..', 'functions/data transforming')))
import config_appdev
import get_eta

In [29]:
gold_standard = pd.read_csv('../data/clean_data/gold_standard.csv')

In [30]:
user_input = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,[4000, 5000],['37.784786', '-122.388954', True]]

In [31]:
# binarize avg_rent column
rent_range = user_input[-2]
gold_standard["rent"][(gold_standard["rent"] < rent_range[0]) | (gold_standard["rent"] > rent_range[1])] = 0
gold_standard["rent"][(gold_standard["rent"] >= rent_range[0]) & (gold_standard["rent"] <= rent_range[1])] = 1
user_input[-2] = 1

/Users/Christine/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/Christine/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [32]:
# binarize most frequent destination input/column
lat_lon = user_input[-1]
if lat_lon[:2] == [0,0]:
    gold_standard["mindist"] = 0
    user_input[-1]=0
else:
    API_KEY = config_appdev.API_KEY
    orig_coord = ','.join(lat_lon[:2])
    eta = get_eta.batch_time(orig_coord,API_KEY,is_driving=lat_lon[2])
    eta["zip"] = eta["zip"].astype(np.int64)
    gold_standard = gold_standard.merge(eta, on="zip", how="left")
    
    eta_25p = np.nanpercentile(gold_standard["mindist"], q=25)
    gold_standard["mindist"][(gold_standard["mindist"] <= eta_25p)] = 1
    gold_standard["mindist"][(gold_standard["mindist"] > eta_25p)] = 0
    user_input[-1]=1

/Users/Christine/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Christine/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [26]:
gold_standard = gold_standard.fillna(0)

# calculate jaccard similarity
for row in range(gold_standard.shape[0]):
    similarity = metrics.jaccard_similarity_score(gold_standard.drop(["similarity", "zip"], axis=1).ix[row,],user_input)
    gold_standard.ix[row,"similarity"] = similarity

In [27]:
gold_standard.sort_values("similarity", ascending=False).reset_index(drop=True).ix[[0,1,2],"zip"]

0    94110
1    94118
2    94111
Name: zip, dtype: int64